This post is inspired by the most fascinating application of PCA I have yet encountered.


Typically, PCA is computed via the eigendecomposition of the scaled sample covariance matrix
\\[
X^T X \in \mathbb R^{k \times k}.
\\]
In the current context, \\(k = 81,000,000\\) and the sample covariance matrix is far too large to explicitly compute. The simplest technique to compute the principle components of \\(X\\) is to instead consider the transpose of the scaled covariance matrix
\\[
XX^T \in \mathbb R^{n \times n}.
\\]
The number of samples \\(n\\) is several orders of magnitude smaller than the dimension of the feature space \\(k\\) and the matrix \\(XX^T\\) easily fits in memory. Also note that \\(\mathbf v \in \mathbb R^n\\) is an eigenvector of \\(XX^T\\) with eigenvalue \\(\lambda\\) if and only if \\(X^T \mathbf v\\) is an eigenvector of \\(X^TX\\) corresponding to the same eigenvalue. In other words, we can calculate the top principal components of \\(X^TX\\) by finding the top principal components of \\(XX^T\\) and left-multiplying by \\(X^T\\). This technique is known as _dual PCA_ and is equivalent to classical multidimensional scaling using Euclidean distance, as described in this [excellent Stack Exchange post](https://stats.stackexchange.com/a/132731) [FIXME]. In the current context, the matrix \\(X\\) is a massive sparse matrix stored in compressed format in 166 separate files and is far too large to fit in memory. To recover the first \\(n\\) eigenvectors of \\(X^TX\\) from the eigenvectors of \\(XX^T\\), we could carry out a piecewise matrix multiplication by iterating through each file. But since I've already computed the pairwise square Euclidean distance matrix \\(D^2\\), I can easily compute the top principal components of \\(X^TX\\) by taking the eigencomposition of
\\[
K = -\left(I_n - \frac{\mathbf 1_n \mathbf 1_n^T}{n}\right) \frac{D^2}{2}\left(I_n - \frac{\mathbf 1_n \mathbf 1_n^T}{n}\right),
\\]
as explained in [FIXME]. Here, \\(\mathbf 1_n \in \mathbb R^n\\) is the so-called "one vector" whose entries are all ones, and \\(\mathbf 1_n \mathbf 1_n^T\\) is the \\(n\times n\\) matrix whose entries are all ones.

In [7]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

x, y, z = np.random.multivariate_normal(np.array([0,0,0]), np.eye(3), 400).transpose()

trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=12,
        color=z,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='3d-scatter-colorscale')